# **(4,5강-실습) RAG with LangChain**

### 실습 목차
1. LangChain을 활용한 LLM Application
 * Zero-shot CoT
 * Summarization
 * Agents   
2. Retrieval Augmented Generation (RAG)

### 실습 개요 및 배경
> LLM 기반 애플리케이션 개발은 단순히 프롬프트를 입력하고 응답을 받는 것 이상의 복잡한 과정을 포함합니다. 본 실습에서는 LLM과 애플리케이션의 통합을 간소화하도록 도와주는 LangChain에 대해서 배워보도록 하겠습니다. 그리고 마지막으로 LangChain을 활용한 Retrieval-Augmented Generation(RAG)을 통해 LLM이 가지고 있는 한계인 Temporal misalignment와 Hallucination을 해결해 보도록 하겠습니다.

### 실습 수행으로 얻을 수 있는 역량
* LangChain을 활용하여 LLM Application을 제작할 수 있다.
* RAG을 통하여 Temporal misalginment와 Hallucination을 완화할 수 있다.

### Required Packages

In [1]:
!pip install "datasets>=2.16.0" "langchain>=0.0.352" "google-generativeai==0.8.3" "tiktoken>=0.5.2" "duckduckgo_search==7.1.1" "faiss-gpu>=1.7.2" "sentence-transformers>=2.2.2" "protobuf>=4.25.2" \
             "transformers>=4.35.2" "accelerate>=0.24.1" "bitsandbytes>=0.41.2" "sentencepiece>=0.1.99" "torch>=2.1.0" "langchain-google-genai" "scikit-learn>=1.2.2" "langchainhub>=0.1.14" "langchain-community"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1

## 1. LangChain을 활용한 LLM Application
LangChain은 대형언어모델(LLM)을 기반으로 애플리케이션을 구축하기 위한 오픈 소스 프레임워크입니다. LangChain은 프롬프트 엔지니어링의 효율성을 높이기 위해 애플리케이션 개발 과정을 간소화합니다. 또한, LangChain은 챗봇, 질문 대답, 콘텐츠 생성, 요약(Summarization) 등 다양한 언어 모델 기반 애플리케이션을 쉽게 개발할 수 있도록 지원합니다.

[LangChain의](https://python.langchain.com/docs/modules/) [구성 요소​](https://python.langchain.com/docs/integrations/components)는 여러가지가 있지만 다음과 같은 대표적인 요소가 있습니다.

* Model Wrapper
 * 다양한 LLM 모델들을 하나의 라인으로 불러올 수 있으며, 크게 2가지 분류로 나눠집니다.
 * [Completion models](https://python.langchain.com/docs/integrations/llms/): 단순히 주어진 Prompt를 완성시키는 모델들입니다. 대표적으로 `gpt-3.5-turbo-instruct`와 일반 `Llama-2` 모델이 있습니다.
 * [Chat models](https://python.langchain.com/docs/integrations/chat/): 채팅 형태로 생성하는 모델들입니다. 대표적으로 `gpt-3.5-turbo`, `gpt-4` 모델이 있습니다.

* Chain
 * Chain은 일종의 호출 시퀀스를 의미하며, 이전 스텝의 아웃풋이 다음 스텝의 인풋 (prompt의 특정 variable)으로 들어가, 시퀀스 단위의 작업을 수행할 수 있게 도와줍니다. Chain은 Prompt나 LLM 등을 엮는 등, 여러 Chain들을 엮어서 복잡한 Chain을 만들 수 있습니다.

* Embeddings and VectorStore
 * 선택한 임베딩 모델로 다큐먼트를 임베딩을 할 수 있고, Retrieval에 활용할 수 있습니다.

* Agents & Tool
 * 실습 2에서 제작하였던 LLM Agent를 쉽게 만들 수가 있으며, 사전에 정의된 함수(예: 계산기)를 호출할 수 있도록 도와줍니다.

이번 실습 시간에서는 LangChain 라이브러리를 활용해서 Gemini 또는 Llama2 기반 LLM Application을 만들어보도록 하겠습니다. LangChain의 Model Wrapper를 사용하면 다양한 LLM 모델들을 하나의 라인으로 불러올 수 있습니다.

> ***이번 실습에서는 Completion 모델만 다룹니다. Chat 모델은 구현 방법이 달라져야하나, 기본적인 원리는 비슷하니 참고바랍니다.***

**Gemini-1.5-pro**

LangChain은 기본적으로 OpenAI 및 Gemini와 같은 서비스에 최적화가 되어 있습니다. Gemini Model을 사용하려면 [여기](https://ai.google.dev/gemini-api/docs?gad_source=1&gclid=Cj0KCQiAyc67BhDSARIsAM95Qzu0dEZ8MJq2E_x7K9VJ7DE7fZ3HdE8-FOkYYg_rTcw9vfsyozx9E_UaAv_DEALw_wcB&hl=ko)에서 API key를 발급받은 뒤 아래와 같이 사용하시면 됩니다.

In [7]:
import getpass
import os


os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
)

**Llama 2**

Open-source 모델을 기반으로하여 LangChain을 활용할 수도 있습니다. Gemini API를 사용할 수 없는 경우, 아래의 Llama 2 모델을 활용해봅시다.

In [38]:
import torch
from transformers import BitsAndBytesConfig
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="NousResearch/Llama-2-13b-chat-hf",
    task="text-generation",
    device_map="auto",
    model_kwargs={
        "quantization_config": BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16
        )
    },
    pipeline_kwargs={
        "max_new_tokens": 1024,
        "do_sample": True,
        "temperature": 0.01,
        "top_p": 0.9,
        "repetition_penalty": 1.15
    }
)

먼저 가장 기본적인 LLM을 활용한 Text 생성을 해보도록 하겠습니다.

LangChain에서 [모델에 따라 다양한 호출 방법](https://python.langchain.com/docs/integrations/llms/)을 사용할 수 있습니다. 일반적으로 호출 가능한 함수는 다음과 같습니다.
* `invoke`: 하나의 입력에 대해 Chain을 호출
* `batch`: 일련의 입력을 배치화하여 Chain을 호출
* `stream`: 하나의 입력에 대해 순차적으로 생성 결과를 반환하는 Chain을 호출

다음과 같이 Text 생성을 llm chain의 invoke, batch, stream으로 생성해봅시다.


In [10]:
llm.invoke("Seoul is the capital city of")

AIMessage(content='South Korea\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-8a3b27da-3d95-479f-a427-9ef847b651b8-0', usage_metadata={'input_tokens': 7, 'output_tokens': 3, 'total_tokens': 10, 'input_token_details': {'cache_read': 0}})

In [11]:
llm.batch(["Seoul is the capital city of", "Tokyo is the capital city of"])

[AIMessage(content='South Korea\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-65f14e86-871a-4c5e-b414-df1adc726295-0', usage_metadata={'input_tokens': 7, 'output_tokens': 3, 'total_tokens': 10, 'input_token_details': {'cache_read': 0}}),
 AIMessage(content='Japan\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-a07a581a-5497-4479-bc50-3acf49e8a4b3-0', usage_metadata={'input_tokens': 7, 'output_tokens': 2, 'total_tokens': 9, 'input_token_details': {'cache_read': 0}})]

In [12]:
# Huggingface Pipeline에선 지원하지 않습니다.
for s in llm.stream("Seoul is the capital city of"):
    print(s, end="", flush=True)

content='South' additional_kwargs={} response_metadata={'safety_ratings': []} id='run-5c2f3f13-d8f8-4474-bded-26c9f1142550' usage_metadata={'input_tokens': 7, 'output_tokens': 0, 'total_tokens': 7, 'input_token_details': {'cache_read': 0}}content=' Korea\n' additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'safety_ratings': []} id='run-5c2f3f13-d8f8-4474-bded-26c9f1142550' usage_metadata={'input_tokens': 0, 'output_tokens': 3, 'total_tokens': 3, 'input_token_details': {'cache_read': 0}}

### **Zero-shot CoT with LangChain**
Prompt를 다루다보면 텍스트를 Formatting이 필요한 경우가 많습니다. LangChain은 다음과 같이 `PromptTemplate`을 만들 수 있습니다.

1강 Reasoning 실습에 사용한 예시를 가져와 보겠습니다.

In [13]:
from langchain.prompts import PromptTemplate

template = """\
Question: {question}
Answer: Let's think step by step.\
"""

prompt = PromptTemplate.from_template(template)

`PromptTemplate`은 앞선 `llm`처럼 `invoke`와 `batch`를 통해 입력이 Formatting된 prompt를 만들 수 있습니다.

In [14]:
prompt.invoke({"question": "The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?"})

StringPromptValue(text="Question: The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?\nAnswer: Let's think step by step.")

In [15]:
prompt.batch([
    {"question": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"},
    {"question": "The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?"}
])

[StringPromptValue(text="Question: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?\nAnswer: Let's think step by step."),
 StringPromptValue(text="Question: The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?\nAnswer: Let's think step by step.")]

LangChain은 이러한 구성요소 들을 다음과 같이 하나의 Chain으로 묶을 수 있습니다.
이러한 방식을 **LangChain Expression Language (LCEL)**이라고 합니다. LECL은 Chain을 쉽게 구성할 수 있는 선언 방식입니다. LCEL은 가장 간단한 "Prompt + LLM" 체인부터 복잡한 기능을 하는 Chain까지 만들 수 있습니다.

In [16]:
llm_chain = prompt | llm

이렇게 묶여서 Chain이 된 요소는 앞서 배운 방식대로 호출할 수가 있습니다.

In [17]:
llm_chain.invoke({"question": "The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?"})

AIMessage(content='1. **Apples consumed per day:** The cafeteria consumes 23 + 37 = 60 apples each day.\n\n2. **Apples consumed in 3 days:** Over 3 days, they consume 60 * 3 = 180 apples.\n\n3. **Apples remaining:**  Starting with 325 apples, and consuming 180, they have 325 - 180 = 145 apples left.\n\nSo the answer is $\\boxed{145}$.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-4215eefe-a2b5-439a-833a-d6234d757ee1-0', usage_metadata={'input_tokens': 50, 'output_tokens': 112, 'total_tokens': 162, 'input_token_details': {'cache_read': 0}})

In [28]:
llm_chain.batch([
    {"question": "The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?"},
    {"question": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"}
])

[AIMessage(content='The cafeteria consumes 23 + 37 = 60 apples each day.\n\nOver 3 days, they consume 60 * 3 = 180 apples.\n\nStarting with 325 apples, after 3 days there will be 325 - 180 = 145 apples left.\n\nSo the answer is $\\boxed{145}$.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-83fd61a3-f23d-436f-9cba-f6d09a193d4c-0', usage_metadata={'input_tokens': 50, 'output_tokens': 85, 'total_tokens': 135, 'input_token_details': {'cache_read': 0}}),
 AIMessage(content='Roger starts with 5 tennis balls.\n\nHe buys 2 cans * 3 balls/can = 6 balls.\n\nHe now has 5 + 6 = 11 tennis balls.\n\nSo the answer is $\\boxed{11}$.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-90450c1c-68ac-41ef-8a68-f784f8e876a0-0', usage_metadata={'input_tokens': 48, 'output_tokens': 53

In [29]:
# Huggingface Pipeline에선 지원하지 않습니다.
for s in llm_chain.invoke({"question": "The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?"}):
    print(s, end="", flush=True)

('content', '1. **Calculate the total apples consumed per day:** 23 apples (lunch) + 37 apples (dinner) = 60 apples/day\n\n2. **Calculate the total apples consumed over 3 days:** 60 apples/day * 3 days = 180 apples\n\n3. **Calculate the remaining apples:** 325 apples (initial) - 180 apples (consumed) = 145 apples\n\nTherefore, there will be 145 apples left after 3 days.\n')('additional_kwargs', {})('response_metadata', {'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []})('type', 'ai')('name', None)('id', 'run-d9e23355-1f33-4d4b-b752-08ee9795d66f-0')('example', False)('tool_calls', [])('invalid_tool_calls', [])('usage_metadata', {'input_tokens': 50, 'output_tokens': 113, 'total_tokens': 163, 'input_token_details': {'cache_read': 0}})

한편 두 구성 요소를 묶을 때, 둘 중 하나의 요소가 LangChain 요소이기만 해도 이들을 Chain으로 묶을 수 있습니다. 즉, 둘 중 하나가 다른 일반적인 함수인 경우에도 이들을 Chain으로 묶는 것이 가능합니다.

Dictionary 형태를 활용하여 두 Chain을 병렬적으로 실행하는 것도 가능합니다.

In [37]:
llm_chain = (
    prompt
    | llm
    | {
        "answer": "{0}\nTherefore, the answer is".format | llm
    }
)

In [38]:
llm_chain.batch([
    {"question": "The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?"},
    {"question": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"}
])

[{'answer': AIMessage(content='145\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-78bfc026-e04a-4c10-948d-b9a2316673f9-0', usage_metadata={'input_tokens': 258, 'output_tokens': 4, 'total_tokens': 262, 'input_token_details': {'cache_read': 0}})},
 {'answer': AIMessage(content='11\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b91546b5-826a-4022-b193-e01e7b38ca5f-0', usage_metadata={'input_tokens': 197, 'output_tokens': 3, 'total_tokens': 200, 'input_token_details': {'cache_read': 0}})}]

`RunnablePassthrough`를 활용하면 값을 그대로 전달하거나 추가하는 것 또한 가능합니다.

In [39]:
# RunnablePassthrough는 Data를 그대로 혹은 kwargs를 추가하여 넘겨주는 역할을 하며,
# RunnablePassthrough.assign을 통해 원하는 함수를 적용할 수 있습니다.
from langchain_core.runnables import RunnablePassthrough

llm_chain = (
    {"question": RunnablePassthrough()}
    | RunnablePassthrough.assign(
        completion=prompt | llm
    )
)

In [40]:
llm_chain.invoke("The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?")

{'question': 'The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?',
 'completion': AIMessage(content='1. **Apples consumed per day:** The cafeteria consumes 23 + 37 = 60 apples per day.\n\n2. **Apples consumed in 3 days:**  Over 3 days, they consume 60 * 3 = 180 apples.\n\n3. **Apples remaining:**  Starting with 325 apples and consuming 180, there will be 325 - 180 = 145 apples left.\n\nTherefore, there will be 145 apples left after 3 days.\nFinal Answer: The final answer is $\\boxed{145}$\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b9bd1339-4924-46f7-b179-fa01c48f6110-0', usage_metadata={'input_tokens': 50, 'output_tokens': 133, 'total_tokens': 183, 'input_token_details': {'cache_read': 0}})}

위의 요소들을 활용하여, 실습1에서의 복잡한 Reasoning Step을 Chain을 구성하여 Prompt를 만드는 것에서부터 답을 추출하는 것 까지 하나의 Chain으로 만들어 줄 수 있습니다.

In [41]:
# 파이썬 표준 라이브러리의 itemgetter를 활용하면,
# Dictionary의 특정 값을 가져오는 함수를 만들 수 있습니다.
from operator import itemgetter
# Langchain은 다양한 Output parser를 지원합니다:
# https://python.langchain.com/docs/modules/model_io/output_parsers/
from langchain.output_parsers.regex import RegexParser

reasoning_template = """\
Question: {question}
Answer: Let's think step by step.\
"""

extraction_template = reasoning_template + """\
{chain_of_thought}
Therefore, the answer is (in arabic numbers)\
"""

reasoning_prompt = PromptTemplate.from_template(reasoning_template)
extraction_prompt = PromptTemplate.from_template(extraction_template)

zero_shot_cot = (
    {"question": RunnablePassthrough()}
    | RunnablePassthrough.assign(
        chain_of_thought = reasoning_prompt | llm,
    )
    | extraction_prompt
    | llm
    | RegexParser(regex=r"(\d+)", output_keys=["answer"])
    | itemgetter("answer")
    | int
)

In [42]:
zero_shot_cot.batch([
    "The cafeteria had 325 apples. If they consume 23 for lunch and 37 for dinner each day, how many will be left after 3 days?",
    "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"
])

[145, 11]

### **Summarization with LangChain**
다음으로는 아래 문서를 요약하는 방법을 배워보겠습니다.
일반적으로 문서 요약은 해당 문서의 전체적인 맥락과 의미에 대한 정확한 이해를 요구하며, 특히 매우 긴 문서를 요약할 때 중요한 정보를 포함시키면서도 간결하게 유지해야 하는 어려운 과제입니다.
저희는 이러한 문서 요약을 LangChain을 활용하여 수행해보도록 하겠습니다.

문서 요약 실습을 위하여 [한국어 Wikipedia](https://ko.wikipedia.org/wiki)의 한 문서를 들고와 봅시다.

In [43]:
import requests

response = requests.get("https://ko.wikipedia.org/w/api.php?action=query&prop=revisions&titles=제1차_서울_전투&rvslots=*&rvprop=content&formatversion=2&format=json")

document = response.json()["query"]["pages"][0]["revisions"][0]["slots"]["main"]["content"]
print(document)

{{위키데이터 속성 추적}}
{{알찬 글}}
{{전쟁 정보
|분쟁 = 제1차 서울 전투
|전체 = [[6.25 전쟁]]
|그림 = 파일:Korean-War-june-aug-1950.png
|설명 = 개전 극초기 [[북한군]]의 진격로
|날짜 = 1950년 6월 26일-28일
|장소 = [[서울특별시|서울]] 및 인근 지역
|결과 = [[조선민주주의인민공화국]]의 승리와 [[서울특별시|서울]] 함락
|교전국1 = {{국기|대한민국|1949|size=23px}}
|교전국2 = {{국기|조선민주주의인민공화국}}
|지휘관1 = {{국기그림|대한민국|1949}} [[이승만]] <br /> {{국기그림|대한민국|1949}} [[채병덕]]
|지휘관2 = {{국기그림|조선민주주의인민공화국}} [[김일성]] <br /> {{국기그림|조선민주주의인민공화국}} [[김책]]
|병력1 = 약 65,000명 (축차 투입 포함)
|병력2 = 약 107,000명
|사상자1 = 전사자 및 실종자 약 44,000명
|사상자2 = 사상자 약 1,112명
}}
{{전역상자 한국 전쟁}}
{{전역상자 제1차 서울 전투}}

'''제1차 서울 전투'''( - 戰鬪, {{문화어|서울 해방}}, {{llang|en|First Battle of Seoul}})는 [[6.25 전쟁]] 초기에 일어났던 전투 중 하나이다. 국경에서의 전투에 이어 1950년 6월 26일부터 28일까지 구 서울시를 중심으로 하여 중부 지역 전범위에서 일어난 [[대한민국]] 국군과 [[조선민주주의인민공화국]]의 조선인민군 사이의 교전이다. 교전 결과 [[조선인민군]]이 서울을 점령하였으며 국군은 [[한강]] 이남 지역으로 밀려났다.

== 배경 ==
{{본문|폭풍 작전}}

1950년 6월 25일 새벽 3시 30분 [[조선인민군]]이 [[T-34]]를 앞에 두고 곡사포로 엄호하는 [[전격전]]의 방식으로 [[38선]] 전선을 월경하면서 [[한국 전쟁]]이 시작되었다. 북한은 [[폭풍 작전]] 구상 때 서울이 함락되면 사실상 승패가

In [44]:
print("문서 토큰 길이:", llm.get_num_tokens(document))

문서 토큰 길이: 15453


위에서 볼 수 있듯이 해당 문서는 매우 긴 길이로 구성되어 있습니다.
이러한 긴 문서를 효과적으로 다루기 위하여, 해당 텍스트를 여러 개의 Text chunks로 나누어봅시다.
이러한 텍스트를 Chunk들로 나누는 연산은 Langchain의 TextSplitter를 활용하면 쉽게 구현할 수 있습니다.


In [45]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2048,          # Llama 2를 사용할 시 500을 사용
    chunk_overlap=20,
    length_function=llm.get_num_tokens,
    separators=["\n\n", "\n", " ", ""]
)

실제로 문서가 잘 나눠지는지 확인해봅시다.

In [46]:
chunks = text_splitter.split_text(document)
print("총 Chunk 개수:", len(chunks))

for chunk in chunks[:3]:
    print('='*100)
    print(chunk)
    print("토큰 길이:", llm.get_num_tokens(chunk))

총 Chunk 개수: 9
{{위키데이터 속성 추적}}
{{알찬 글}}
{{전쟁 정보
|분쟁 = 제1차 서울 전투
|전체 = [[6.25 전쟁]]
|그림 = 파일:Korean-War-june-aug-1950.png
|설명 = 개전 극초기 [[북한군]]의 진격로
|날짜 = 1950년 6월 26일-28일
|장소 = [[서울특별시|서울]] 및 인근 지역
|결과 = [[조선민주주의인민공화국]]의 승리와 [[서울특별시|서울]] 함락
|교전국1 = {{국기|대한민국|1949|size=23px}}
|교전국2 = {{국기|조선민주주의인민공화국}}
|지휘관1 = {{국기그림|대한민국|1949}} [[이승만]] <br /> {{국기그림|대한민국|1949}} [[채병덕]]
|지휘관2 = {{국기그림|조선민주주의인민공화국}} [[김일성]] <br /> {{국기그림|조선민주주의인민공화국}} [[김책]]
|병력1 = 약 65,000명 (축차 투입 포함)
|병력2 = 약 107,000명
|사상자1 = 전사자 및 실종자 약 44,000명
|사상자2 = 사상자 약 1,112명
}}
{{전역상자 한국 전쟁}}
{{전역상자 제1차 서울 전투}}

'''제1차 서울 전투'''( - 戰鬪, {{문화어|서울 해방}}, {{llang|en|First Battle of Seoul}})는 [[6.25 전쟁]] 초기에 일어났던 전투 중 하나이다. 국경에서의 전투에 이어 1950년 6월 26일부터 28일까지 구 서울시를 중심으로 하여 중부 지역 전범위에서 일어난 [[대한민국]] 국군과 [[조선민주주의인민공화국]]의 조선인민군 사이의 교전이다. 교전 결과 [[조선인민군]]이 서울을 점령하였으며 국군은 [[한강]] 이남 지역으로 밀려났다.

== 배경 ==
{{본문|폭풍 작전}}

1950년 6월 25일 새벽 3시 30분 [[조선인민군]]이 [[T-34]]를 앞에 두고 곡사포로 엄호하는 [[전격전]]의 방식으로 [[38선]] 전선을 월경하면서 [[한국 전쟁]]이 시작되었다. 북한은 [[폭풍 작전]] 구상 때 서울

다음으로 나눠진 Text Chunks를 활용하여 MapReduce를 통한 Summarization을 확인해보겠습니다. LangChain의 MapReduce는 먼저 작은 Text Chunk를 요약한 다음, 요악된 내용을 하나의 요약으로 결합하여 Summarization을 수행하게 됩니다.

In [54]:
from langchain_core.runnables import RunnableLambda

map_prompt_template = """\
Context:
{text}

Question: Write a summary of above chunk of text that includes the main points and any important details in a single paragraph.
Answer:\
"""

combine_prompt_template = """\
Write a concise summary of the following text delimited by triple backquotes.
Return your response in bullet points which covers the key points of the text.
```
{text}
```
BULLET POINT SUMMARY:
"""

map_prompt = PromptTemplate.from_template(map_prompt_template)
combine_prompt = PromptTemplate.from_template(combine_prompt_template)

summarize = (
    # Chain으로 엮기 위해선 처음 나오는 두 요소 중 최소한 하나는 LangChain 요소이어야만 합니다.
    # 그러나 "text_splitter.split_text"과 "itemgetter(slice(3))"는 모두 평범한 함수이므로,
    # 둘 중 하나를 RunnableLambda로 감싸서 LangChain 요소로 명시적으로 바꿉니다.
    RunnableLambda(text_splitter.split_text)
    | itemgetter(slice(3))                    # 빠른 실행을 위해 가장 앞쪽 3개의 분할 문서만 활용해 봅시다.
    | ({"text": RunnablePassthrough()} | map_prompt | llm).batch
    | {
            "text": lambda messages: "\n".join(
                message.content if hasattr(message, "content") else str(message)
                for message in messages
            )
        }
    | combine_prompt
    | llm
)

In [55]:
print(summarize.invoke(document))

content="* **Initial NKPA Advance (June 25-26):** The North Korean People's Army (NKPA) launched a surprise attack across the 38th parallel, aiming to quickly capture Seoul. The Republic of Korea Army (ROKA), outgunned and outnumbered, was pushed back despite valiant resistance, including close-quarters attacks against T-34 tanks.\n\n* **Battles Along the Imjin River (June 26-28):**  The ROKA 1st Infantry Division held defensive positions along the Imjin River.  Initial South Korean counterattacks were successful, but the fall of Uijeongbu exposed their flank, forcing a retreat to the Bongilcheon Line.  Despite repelling several attacks, the fall of Miryang Ridge in Seoul cut off their rear, leading to a retreat and subsequent battles.\n\n* **Failed South Korean Counteroffensive (June 26-28):** A planned South Korean counteroffensive near Uijeongbu failed due to the late arrival of reinforcements, with only limited forces from the 2nd and 7th Divisions available.  Concerns from divisio

## 2. LangChain Agents

다음으로 LangChain으로 Agents를 구현해보도록 하겠습니다. Agents는 의사 결정권자로서 질문을 보고, 다음에 취해야 할 Action을 추론하고, 사전에 정의된 Tool을 활용하여 해당 Action 실행하게 됩니다.
2강 실습에서 사용했던 DuckDuckGoSearch를 Tool로 활용하여 질문에 대한 답을 해결해보겠습니다.

먼저 Tool로서 사용할 DuckDuckGoSearchRun API를 살펴보겠습니다.

In [56]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("What is the Narita International Airport Corporation?")

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


"Before Narita opened, Tokyo International Airport (also known as Haneda Airport) was Tokyo's main international airport.Haneda, located in Tokyo Bay was surrounded by densely populated residential and industrial areas, and began to suffer capacity and noise issues in the early 1960s as jet aircraft became common. The Japanese transport ministry commissioned a study of alternate airport ... Narita International Airport (IATA: NRT) is one of Japan's primary international gateways, serving as a critical hub for both passengers and cargo in the Asia-Pacific region. Located in Narita, Chiba prefecture, approximately 60 kilometers east of central Tokyo, the airport was constructed to alleviate congestion at the older Haneda Airport ... Narita International Airport (NRT) is located about 60 kilometers east of central Tokyo. It focuses primarily on international flights, serving over 40 international airlines and connecting over 100 destinations worldwide. In 2019, it handled approximately 44

이제 DuckDuckGoSearchRun를 Tool로서 LLM이 활용하여 답변을 생성하는 과정을 살펴보겠습니다.

이번 실습에서는 ReAct를 Agent로 사용해보도록 하겠습니다. Langchain에도 [Hub](https://smith.langchain.com/hub)가 존재하여 다양한 Prompt와 Chain을 다운 받을 수 있습니다.

In [57]:
from langchain import hub

prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


먼저 ReACT의 Prompt를 살펴보면, 아래의 변수들이 Prompt에 들어가게 됩니다.

In [58]:
print("프롬프트 내 변수", prompt.input_variables)
print("프롬프트 완성 예시:", prompt.invoke({
    'input': "Agent 입력",
    'agent_scratchpad': "스크래치 패드",
    'tool_names': ["도구 1", "도구 2"],
    'tools': ["도구 1: 설명", "도구 2: 설명"]
}).text, sep="\n")

프롬프트 내 변수 ['agent_scratchpad', 'input', 'tool_names', 'tools']
프롬프트 완성 예시:
Answer the following questions as best you can. You have access to the following tools:

['도구 1: 설명', '도구 2: 설명']

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [['도구 1', '도구 2']]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Agent 입력
Thought:스크래치 패드


LangChain은 이러한 Agent를 다음과 같이 쉽게 작동시킬 수 있습니다.

In [59]:
from langchain.agents import AgentExecutor, create_react_agent

tools = [search]
agent = create_react_agent(llm, tools, prompt)

executor = AgentExecutor(agent=agent, tools=tools, max_iteration=3)

In [60]:
executor.invoke({"input": "Tell me the detailed information of the Narita International Airport Corporation."})

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py

{'input': 'Tell me the detailed information of the Narita International Airport Corporation.',
 'output': "Detailed information on the Narita International Airport Corporation (NAA) can be found by exploring the investor relations section of their official website.  This section will likely include financial reports, corporate governance information, and other relevant details about the company's structure and performance."}

위와 같이 LLM이 다음에 수행해야 할 Action이 무엇인지 추론하고, 검색 Tool을 사용하여 답을 구하는 과정을 볼 수 있습니다.

## 3. Retrieval Augmented Generation

이제 External Knowledge Base에서 관련 정보를 검색하여 해당 정보를 LLM에게 제공하여 답을 생성하는 Retrieval Augmented Generation를 구현해 보겠습니다.

**Loading Data**

이번 실습에서는 Wikipedia dataset을 활용합니다.
* Wikipedia dataset은 [Wikimedia](https://dumps.wikimedia.org/)에서 제공하는 Wikipedia dump로부터 추출하여 정제된 dataset입니다.
* 해당 데이터셋은 각 년도까지의 Wikipedia 정보를 여러 개의 언어로 제공됩니다.

In [61]:
from datasets import load_dataset

data = load_dataset("wikimedia/wikipedia", "20231101.ko")

README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/400M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/205M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/177M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/647897 [00:00<?, ? examples/s]

본 실습에서는 해당 데이터셋 중 2000개의 문서만을 사용해봅시다.

In [62]:
data = data['train'][-2000:]

다음과 같이 각 문서는 한국어로 이루어진 정보로 이루어져 있음을 확인할 수 있습니다.

In [63]:
document = data['text'][1059]
print(document)

2023 코파 델 레이 결승전()은 스페인의 최고 축구 컵대회의 120번째 시즌인 코파 델 레이 2022-23의 우승 구단을 결정하기 위해 진행된 경기이다. 이 경기는 2023년 5월 6일, 세비야의 라 카르투하에서 열렸고, 레알 마드리드와 오사수나가 경합했다.

이 대회를 이전까지 19번 우승한 레알 마드리드는 2014년 이래 첫 결승전에 올랐다. 한편 오사수나는 준우승을 거둔 2005년이 유일한 결승전 경험이었다.

레알 마드리드는 이 경기에서 2-1로 이기며 7년 만이자 통산 20번째 코파 델 레이 트로피를 챙겼다.

결승전까지의 경기 

범례: (안) = 안방 경기; (원) = 원정 경기

경기

상세 경기 정보

참고

각주

외부 링크 

2023
코파 델 레이 결승전 2023
코파 델 레이 결승전 2023
코파 델 레이 결승전 2023


### **Retrieval**
임베딩 모델은 Open source인 [Korean Sentence Bert](https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS)를 사용해 볼 수 있습니다.

**Korean Sentence Bert**

In [64]:
from langchain.embeddings import SentenceTransformerEmbeddings

embedder = SentenceTransformerEmbeddings(model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS")

실제로 Embedding이 잘되는지 확인해봅시다.

In [65]:
from sklearn.metrics.pairwise import cosine_similarity

doc_ref = "대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다."
doc_1 = "대한민국은 민주공화국이다."
doc_2 = "치맥은 대한민국의 국민 간식이다."

doc_ref, doc_1, doc_2 = embedder.embed_documents([doc_ref, doc_1, doc_2])

print("Reference와 첫번째 문서의 유사도:", cosine_similarity([doc_ref], [doc_1])[0, 0])
print("Reference와 두번째 문서의 유사도:", cosine_similarity([doc_ref], [doc_2])[0, 0])

Reference와 첫번째 문서의 유사도: 0.5703777653818457
Reference와 두번째 문서의 유사도: 0.23265355874629357


이제 우리가 가진 Data를 Faiss를 사용하여 External Knowledge Base로 만들어보겠습니다. LangChain에는 Faiss가 내장되어 있으며, 이를 활용하여 우리는 쉽게 Data와 Meta 정보를 함께 저장할 수 있습니다.

In [66]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20,
    separators=["\n\n", "\n", " ", ""]
)

texts = []
metadatas = []

for id, url, title, text in zip(data['id'], data['url'], data['title'], data['text']):
    # 해당 record의 메타 데이터를 만들어줍니다.
    metadata = {
        'wiki-id': id,
        'source': url,
        'title': title
    }
    # Document를 여러 text chunk로 쪼갠 후 각 chunk마다 같은 메타 데이터를 저장해 줍니다.
    record_texts = text_splitter.split_text(text)
    record_metadatas = [{
        "chunk": j, "text": chunk_text, **metadata
    } for j, chunk_text in enumerate(record_texts)]

    texts.extend(record_texts)
    metadatas.extend(record_metadatas)

In [67]:
from langchain import FAISS

vectorstore = FAISS.from_texts(texts, embedder, metadatas)

만들어진 벡터 저장소를 활용하면 저장된 Document중 사용자의 Query와 가장 유사한 Document를 찾을 수 있습니다.

In [68]:
query = "2023 코파 델 레이 결승전은 팀이 우승했나요?"

vectorstore.similarity_search_with_score(query, k=3)

[(Document(id='ff0a293c-9cba-4ab5-b7f9-9e1df76ba852', metadata={'chunk': 0, 'text': '2023 코파 델 레이 결승전()은 스페인의 최고 축구 컵대회의 120번째 시즌인 코파 델 레이 2022-23의 우승 구단을 결정하기 위해 진행된 경기이다. 이 경기는 2023년 5월 6일, 세비야의 라 카르투하에서 열렸고, 레알 마드리드와 오사수나가 경합했다.\n\n이 대회를 이전까지 19번 우승한 레알 마드리드는 2014년 이래 첫 결승전에 올랐다. 한편 오사수나는 준우승을 거둔 2005년이 유일한 결승전 경험이었다.\n\n레알 마드리드는 이 경기에서 2-1로 이기며 7년 만이자 통산 20번째 코파 델 레이 트로피를 챙겼다.\n\n결승전까지의 경기 \n\n범례: (안) = 안방 경기; (원) = 원정 경기\n\n경기\n\n상세 경기 정보\n\n참고\n\n각주\n\n외부 링크 \n\n2023\n코파 델 레이 결승전 2023\n코파 델 레이 결승전 2023\n코파 델 레이 결승전 2023', 'wiki-id': '3705509', 'source': 'https://ko.wikipedia.org/wiki/2023%20%EC%BD%94%ED%8C%8C%20%EB%8D%B8%20%EB%A0%88%EC%9D%B4%20%EA%B2%B0%EC%8A%B9%EC%A0%84', 'title': '2023 코파 델 레이 결승전'}, page_content='2023 코파 델 레이 결승전()은 스페인의 최고 축구 컵대회의 120번째 시즌인 코파 델 레이 2022-23의 우승 구단을 결정하기 위해 진행된 경기이다. 이 경기는 2023년 5월 6일, 세비야의 라 카르투하에서 열렸고, 레알 마드리드와 오사수나가 경합했다.\n\n이 대회를 이전까지 19번 우승한 레알 마드리드는 2014년 이래 첫 결승전에 올랐다. 한편 오사수나는 준우승을 거둔 2005년이 유일한 결승전 경험이었다.\n\n레알 마드리드는 이 경기에서 2-1로 이기며 7년

벡터 저장소에서 Retriever를 생성하여 `invoke`와 `batch`를 호출할 수 있으며, 이를 통해 Chain으로 만드는 것이 가능합니다.

In [69]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

retriever.invoke(query)

[Document(id='ff0a293c-9cba-4ab5-b7f9-9e1df76ba852', metadata={'chunk': 0, 'text': '2023 코파 델 레이 결승전()은 스페인의 최고 축구 컵대회의 120번째 시즌인 코파 델 레이 2022-23의 우승 구단을 결정하기 위해 진행된 경기이다. 이 경기는 2023년 5월 6일, 세비야의 라 카르투하에서 열렸고, 레알 마드리드와 오사수나가 경합했다.\n\n이 대회를 이전까지 19번 우승한 레알 마드리드는 2014년 이래 첫 결승전에 올랐다. 한편 오사수나는 준우승을 거둔 2005년이 유일한 결승전 경험이었다.\n\n레알 마드리드는 이 경기에서 2-1로 이기며 7년 만이자 통산 20번째 코파 델 레이 트로피를 챙겼다.\n\n결승전까지의 경기 \n\n범례: (안) = 안방 경기; (원) = 원정 경기\n\n경기\n\n상세 경기 정보\n\n참고\n\n각주\n\n외부 링크 \n\n2023\n코파 델 레이 결승전 2023\n코파 델 레이 결승전 2023\n코파 델 레이 결승전 2023', 'wiki-id': '3705509', 'source': 'https://ko.wikipedia.org/wiki/2023%20%EC%BD%94%ED%8C%8C%20%EB%8D%B8%20%EB%A0%88%EC%9D%B4%20%EA%B2%B0%EC%8A%B9%EC%A0%84', 'title': '2023 코파 델 레이 결승전'}, page_content='2023 코파 델 레이 결승전()은 스페인의 최고 축구 컵대회의 120번째 시즌인 코파 델 레이 2022-23의 우승 구단을 결정하기 위해 진행된 경기이다. 이 경기는 2023년 5월 6일, 세비야의 라 카르투하에서 열렸고, 레알 마드리드와 오사수나가 경합했다.\n\n이 대회를 이전까지 19번 우승한 레알 마드리드는 2014년 이래 첫 결승전에 올랐다. 한편 오사수나는 준우승을 거둔 2005년이 유일한 결승전 경험이었다.\n\n레알 마드리드는 이 경기에서 2-1로 이기며 7년 

### **Retrieval-Augmented Generation**

이제 External Knowledge Base에서 Query와 관련된 정보를 찾은 다음 LLM에게 해당 정보를 주는 과정을 살펴보겠습니다.

먼저 External Knowledge 없이 답변을 생성해봅시다.

In [70]:
naive_qa_template = """\
Question: {question}
Answer:\
"""
naive_qa_prompt = PromptTemplate.from_template(naive_qa_template)

naive_qa = (
    {"question": RunnablePassthrough()}
    | naive_qa_prompt
    | llm
)

In [71]:
print(naive_qa.invoke("2023 코파 델 레이 결승전은 어느 팀이 우승했나요?"))

content='2023 코파 델 레이 결승전은 **레알 마드리드**가 우승했습니다.\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-b3d47b58-ab73-46e2-b3f6-ba45f4091a3f-0' usage_metadata={'input_tokens': 31, 'output_tokens': 27, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}}


이제 External Knowledge를 제공했을 때 답변이 어떻게 바뀌는지 알아보겠습니다.

다음은 retriever를 통해 관련있는 여러 개의 document를 찾은 후, 찾은 document들을 이어서 prompt template을 완성시키고 답을 하는 Chain을 구성하는 과정입니다.

In [72]:
rag_qa_template = """\
Answer the question based only on the following context:
{context}

Question: {question}
Answer:\
"""
rag_qa_template = PromptTemplate.from_template(rag_qa_template)

rag_qa = (
    {
        "context": retriever | (lambda docs: "\n\n".join(doc.page_content for doc in docs)),
        "question": RunnablePassthrough()
    }
    | rag_qa_template
    | llm
)

In [73]:
print(rag_qa.invoke("2023 코파 델 레이 결승전은 어느 팀이 우승했나요?"))

content='레알 마드리드가 2-1로 오사수나를 이기고 우승했습니다.\n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-780eb468-9975-48e7-85ae-f8d8b4cf07de-0' usage_metadata={'input_tokens': 812, 'output_tokens': 25, 'total_tokens': 837, 'input_token_details': {'cache_read': 0}}


메타데이터를 활용하여 LLM이 질문에 답하는 데 사용하는 정보의 소스를 포함할 수도 있습니다.

In [74]:
from operator import attrgetter

rag_qa_from_doc = (
    {
        "question": itemgetter("question"),
        "context": RunnableLambda(itemgetter("docs")) | (lambda docs: "\n\n".join(doc.page_content for doc in docs))
    }
    | rag_qa_template
    | llm
)

rag_qa_with_ref = (
    {"question": RunnablePassthrough()}
    | RunnablePassthrough.assign(docs=itemgetter("question") | retriever)
    | {
        "question": itemgetter("question"),
        "answer": rag_qa_from_doc,
        "source": (
            RunnableLambda(itemgetter("docs"))
            | RunnableLambda(lambda doc: doc.metadata["source"]).batch
        )
    }
)

In [75]:
rag_qa_with_ref.invoke("2023 코파 델 레이 결승전은 어느 팀이 우승했나요?")

{'question': '2023 코파 델 레이 결승전은 어느 팀이 우승했나요?',
 'answer': AIMessage(content='레알 마드리드가 2-1로 오사수나를 이기고 우승했습니다.\n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-194f1af7-746a-4660-a84f-55b5cc01a526-0', usage_metadata={'input_tokens': 812, 'output_tokens': 25, 'total_tokens': 837, 'input_token_details': {'cache_read': 0}}),
 'source': ['https://ko.wikipedia.org/wiki/2023%20%EC%BD%94%ED%8C%8C%20%EB%8D%B8%20%EB%A0%88%EC%9D%B4%20%EA%B2%B0%EC%8A%B9%EC%A0%84',
  'https://ko.wikipedia.org/wiki/%EB%9D%BC%EB%A7%88%20%28%EC%BD%9C%EB%A1%9C%EB%9D%BC%EB%8F%84%EC%A3%BC%29',
  'https://ko.wikipedia.org/wiki/%EC%8B%9C%EB%B2%94%20%EA%B2%BD%EA%B8%B0']}

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.